In [ ]:
!pip install pandas==1.3.4 networkx==2.6.3 pyvis==0.1.9
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk import FreqDist
import re
import networkx as nx
from pyvis.network import Network

base = tt_perfil[tt_perfil['Apelido'] == 'Bolsonaro'] #recorte da base com somente um candidato
variavel = 'text' #variável de texto da base
nomearquivo = 'arquivo.html'
nomecsv = 'arquivo.csv'
cor = '#3d70e0'

base[variavel] = base[variavel].astype(str)
listatokens = []

for row in base.index:
  #tokenização iterando por cada linha da base de dados para agrupar os n-gramas por post
  stopwords = set(nltk.corpus.stopwords.words('portuguese'))
  excluidas = {'pra', 'vai', 'ter', 'aqui', 'isso', 'cada', 'vou', 'vão',
               'fazer', 'dar', 'pode', 'todo', 'porque', 'desde', 'boa', 'bom',
               'todos', 'quer', 'ainda', 'após', 'ricardo', 'stuckert', 'set', 'ton',
               'the', 'nan', 'além', 'ricardostuckert', 'set', 'fez', 'mil', 'sobre', 'milhões',
               'assim', 'nesse', 'nos', 'https', 'pro', 'http', 'share_device_id', 'www'}
  dicionario_replaces = {'espírito santo':'espírito_santo', 'espirito santo':'espirito_santo', 'mato grosso do sul':'mato_grosso_do_sul', 'mato grosso':'mato_grosso', 'rio de janeiro':'rio_de_janeiro',
  'rio grande do norte':'rio_grande_do_norte', 'rio grande do sul':'rio_grande_do_sul', 'santa catarina':'santa_catarina', 'são paulo':'são_paulo', 'sao paulo':'sao_paulo','distrito federal':'distrito_federal',
  'rio branco':'rio_branco', 'são luís':'são_luís', 'são luis':'são_luis', 'sao luis':'sao_luis', 'sao luís':'sao_luís', 'campo grande':'campo_grande', 'belo horizonte':'belo_horizonte', 'joão pessoa':'joão_pessoa',
  'joao pessoa':'joao_pessoa', 'porto alegre':'porto_alegre', 'porto velho':'porto_velho', 'boa vista':'boa_vista', 'bom dia':'bom_dia', 'boa tarde':'boa_tarde', 'boa noite':'boa_noite', 'boa semana':'boa_semana',
  'minas gerais':'minas_gerais'}
  base[variavel] = base[variavel].str.lower()
  for key, value in dicionario_replaces.items():
       base[variavel] = base[variavel].str.replace(key, value)  
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  tokenspbi = tokenizer.tokenize(base[variavel][row])
  tokenspbi = [i for i in tokenspbi if (len(i) > 2)]
  tokenspbi = [i for i in tokenspbi if i  not in excluidas]
  tokenspbi = [i for i in tokenspbi if i  not in stopwords]
  tokenspbi = [i for i in tokenspbi if (i.isnumeric() != True)]
  bigramas = list(nltk.bigrams(tokenspbi)) #para outros n-gramas, usar: list(nltk.ngrams(tokenspbi, n))
  listatokens.append(bigramas)

#criar df dos bigramas e suas frequencias
bigramascorridos = [item for sublist in listatokens for item in sublist]
df_bigramas = pd.Series(bigramascorridos).value_counts().to_frame().reset_index()
df_bigramas.columns = ['bigrama', 'frequencia']
df_bigramas[['b1', 'b2']] = pd.DataFrame(df_bigramas['bigrama'].tolist(), index=df_bigramas.index) #para outros n-gramas, criar n colunas

#top tokens que ta no gráfico de frequência
base_tokens = tokenizar(base, variavel)
frequencia = FreqDist(base_tokens)
frequenciasr = pd.Series(dict(frequencia.most_common(30)))
top_words = frequenciasr.to_frame().reset_index()
top_words.columns = ['token', 'frequencia']
toptokens = top_words['token'].tolist()
toptokens = '|'.join(map(str, toptokens))

#filtrando somente os bigramas que são formados por tokens de top frquencia
df_bigramas['b1top'] = df_bigramas['b1'].str.contains("(?<!.)(" + toptokens + ")(?!.)", flags=re.IGNORECASE)
df_bigramas['b2top'] = df_bigramas['b2'].str.contains("(?<!.)(" + toptokens + ")(?!.)", flags=re.IGNORECASE)
df_bigramas['toptokens'] = df_bigramas['b1top'] + df_bigramas['b2top'] #para outros n-gramas, somar n colunas
df_bigramas = df_bigramas[df_bigramas['toptokens'] == True]

### gráfico

#filtrando os bigramas com base em um corte: top 85 e frequência > 1
df_bigramas = df_bigramas.head(85)
df_bigramasrecorte = df_bigramas[df_bigramas['frequencia'] > 1]

d = df_bigramasrecorte.set_index('bigrama').T.to_dict('records')
G = nx.Graph()

#criando as conexões entre os nós
for k, v in d[0].items():
    G.add_edge(k[0], k[1], weight= (v))

pos = nx.spring_layout(G, k=3)

#plotando nodes e edges
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos)

#criando a visualização interativa
net = Network(height='750px', width='100%', bgcolor='white', font_color='black')

for node in G.nodes():
    net.add_node(node, title=node, color = cor, size=15)
for edge in G.edges():
    net.add_edge(edge[0], edge[1], value=G[edge[0]][edge[1]]['weight'])

#cor e fonte dos nós
for n in net.nodes:
    n['font'] = {'size': 20, 'color': '#000000'}

net.show(nomearquivo)
df_bigramas.to_csv(nomecsv)